dependencies </n>
Env :: Forecastinit
Python=3.11.5

Libraries

In [125]:
# General 
import pandas as pd
import numpy as np

# sklearn 
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# skforecast 
from skforecast.datasets import load_demo_dataset
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.model_selection import backtesting_forecaster
from skforecast.model_selection import grid_search_forecaster

Input

In [126]:
Historical_data_org = pd.read_csv('Input/Actual.csv')
TimeKey_org = pd.read_csv('Input/Time.csv')
Forecast_level = ['Version.[Version Name]', 'Channel.[Channel]', 'Account.[Account]',
       'PnL.[PnL]', 'Demand Domain.[Demand Domain]', 'Region.[Region]',
       'Location.[Location]', 'Time.[Planning Month]', 'Item.[Item]']
Time_column = ['Time.[Planning Month]']
Historical_data_column = ['Actual']
#Date should be in format of time key column, which is available in TimeKey file 
Time_key_column_name = "Time.[PlanningMonthKey]"
Time_key_date_format = "%m/%d/%Y %I:%M:%S %p"
Historic_start_date = "M09-20" 
Historic_end_date = "M12-22"
Forecast_start_date = "M01-23"
Forecast_end_date = "M03-23"
drivers = ['driver_Month']


Processing Input

In [127]:
Forecast_level_minus_time = Forecast_level.copy()
Forecast_level_minus_time.remove(Time_column[0])
Forecast_level_minus_time

['Version.[Version Name]',
 'Channel.[Channel]',
 'Account.[Account]',
 'PnL.[PnL]',
 'Demand Domain.[Demand Domain]',
 'Region.[Region]',
 'Location.[Location]',
 'Item.[Item]']

Copy Input 

In [128]:
Historical_data = Historical_data_org.copy(deep=True)
TimeKey = TimeKey_org.copy(deep=True)
TimeKey['TimeKey'] = pd.to_datetime(TimeKey[Time_key_column_name], format=Time_key_date_format)

In [129]:
# calculating Historic_end_date_key
Historic_start_date_key = pd.to_datetime(TimeKey[TimeKey[Time_column[0]]==Historic_start_date]['TimeKey'].values[0])
Historic_end_date_key = pd.to_datetime(TimeKey[TimeKey[Time_column[0]]==Historic_end_date]['TimeKey'].values[0])
Forecast_start_date_key = pd.to_datetime(TimeKey[TimeKey[Time_column[0]]==Forecast_start_date]['TimeKey'].values[0])
Forecast_end_date_key = pd.to_datetime(TimeKey[TimeKey[Time_column[0]]==Forecast_end_date]['TimeKey'].values[0])

Processing Copy Input

In [130]:
Historical_data[Historical_data_column] = Historical_data[Historical_data_column].astype(float)


Creating Key in Historical Data

In [131]:
# creating keys 
Historical_data['key'] = Historical_data[Forecast_level_minus_time].astype(str).agg("__ MDJoinner__".join, axis=1)
# dropping columns, which is already present in keys 
Historical_data.drop(Forecast_level_minus_time,axis=1,inplace=True) 
Historical_data.head(2)

,Time.[Planning Month],Actual,key
0,M07-20,445.0,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...
1,M08-20,711.0,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...


Custom Filter

In [132]:
Historical_data = Historical_data[Historical_data['key']=='CurrentWorkingView__ MDJoinner__B2B__ MDJoinner__AMO__ MDJoinner__DP__ MDJoinner__DP__ MDJoinner__ShipTo1__ MDJoinner__DP__ MDJoinner__Loctite 248 19g Stick']
# Historical_data['key'].values

Merging Time Key in Historical Data

In [133]:
Historical_data = pd.merge(Historical_data,TimeKey[Time_column+["TimeKey"]],on=Time_column,how='left')
Historical_data.head(2)

,Time.[Planning Month],Actual,key,TimeKey
0,M07-20,445.0,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...,2020-07-05
1,M08-20,711.0,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...,2020-08-02


In [134]:
# Creating drivers 
Historical_data['driver_Month'] = Historical_data['TimeKey'].dt.month
Historical_data.head()

,Time.[Planning Month],Actual,key,TimeKey,driver_Month
0,M07-20,445.0,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...,2020-07-05,7
1,M08-20,711.0,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...,2020-08-02,8
2,M09-20,462.0,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...,2020-08-30,8
3,M10-20,174.0,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...,2020-10-04,10
4,M11-20,179.0,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...,2020-11-01,11


Train Data

In [135]:
train = Historical_data[(Historical_data['TimeKey']>=Historic_start_date_key)&(Historical_data['TimeKey']<=Historic_end_date_key)]
X_train = train[['key','TimeKey']+drivers]
y_train = train[[Historical_data_column[0]]]

Test Data

In [136]:
test = Historical_data[(Historical_data['TimeKey']>=Forecast_start_date_key)&(Historical_data['TimeKey']<=Forecast_end_date_key)]
X_test = test[['key','TimeKey']+drivers]
y_test = test[[Historical_data_column[0]]]

In [140]:
forecaster = ForecasterAutoreg(
                 regressor = RandomForestRegressor(random_state=123),
                 lags      = [1]
             )

forecaster.fit(y=y_train[Historical_data_column[0]], exog=X_train[drivers])
forecaster

/home/mdbahauddinlinux/anaconda32023/envs/Forecastinit/lib/python3.11/site-packages/skforecast/utils/utils.py:815: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
/home/mdbahauddinlinux/anaconda32023/envs/Forecastinit/lib/python3.11/site-packages/skforecast/utils/utils.py:940: UserWarning: `exog` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
/home/mdbahauddinlinux/anaconda32023/envs/Forecastinit/lib/python3.11/site-packages/skforecast/utils/utils.py:815: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(


ForecasterAutoreg 
Regressor: RandomForestRegressor(random_state=123) 
Lags: [1] 
Transformer for y: None 
Transformer for exog: None 
Window size: 1 
Weight function included: False 
Differentiation order: None 
Exogenous included: True 
Type of exogenous variable: <class 'pandas.core.frame.DataFrame'> 
Exogenous variables names: ['driver_Month'] 
Training range: [0, 27] 
Training index type: RangeIndex 
Training index frequency: 1 
Regressor parameters: {'bootstrap': True, 'ccp_alpha': 0.0, 'criterion': 'squared_error', 'max_depth': None, 'max_features': 1.0, 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': 123, 'verbose': 0, 'warm_start': False} 
fit_kwargs: {} 
Creation date: 2024-01-22 19:54:25 
Last fit date: 2024-01-22 19:54:25 
Skforecast version: 0.11.0 
Python version: 3.11.5 
Forecaster id: None 

In [161]:
# Predict intervals
predictions = forecaster.predict_interval(steps=1, exog=X_test[drivers+['TimeKey']].set_index('TimeKey'))
# predictions.head(1)

/home/mdbahauddinlinux/anaconda32023/envs/Forecastinit/lib/python3.11/site-packages/skforecast/utils/utils.py:877: UserWarning: `last_window` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
/home/mdbahauddinlinux/anaconda32023/envs/Forecastinit/lib/python3.11/site-packages/skforecast/utils/utils.py:929: UserWarning: `exog` has DatetimeIndex index but no frequency. Index is overwritten with a RangeIndex of step 1.
  warnings.warn(


UnboundLocalError: cannot access local variable 'new_index' where it is not associated with a value

In [160]:
X_test
# drivers
# X_test[drivers]
X_test[drivers+['TimeKey']].set_index('TimeKey')

,driver_Month
TimeKey,
2023-01-01,1
2023-01-29,1
2023-02-26,2


In [149]:
y_train[Historical_data_column[0]]

2     462.0
3     174.0
4     179.0
5     231.0
6     126.0
7     212.0
8     231.0
9     135.0
10    182.0
11    288.0
12    431.0
13    693.0
14    473.0
15    148.0
16    188.0
17    241.0
18    228.0
19    246.0
20    209.0
21    167.0
22    175.0
23    232.0
24    540.0
25    732.0
26    627.0
27    133.0
28    153.0
29    194.0
Name: Actual, dtype: float64

In [150]:
X_train[drivers]

,driver_Month
2,8
3,10
4,11
5,11
6,1
7,1
8,2
9,4
10,5
11,5
